In [ ]:
from tensorflow import keras
from keras.layers import Dense, SimpleRNN, Dropout, Input, Flatten, LSTM, Embedding, Bidirectional
import numpy as np
import pandas as pd
import ast
import re

In [ ]:
import gensim.downloader as api
model_embedding = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
model_embedding.most_similar(positive=['имущество'])

[('арестовал', 0.6874980926513672),
 ('делит', 0.6332550048828125),
 ('расходы', 0.6277076005935669),
 ('гранты', 0.6274681091308594),
 ('ребёнка', 0.6251256465911865),
 ('владельца', 0.6227707862854004),
 ('адвоката', 0.6206133961677551),
 ('отбирать', 0.6170918345451355),
 ('арестовали', 0.6161015033721924),
 ('арест', 0.6145174503326416)]

In [ ]:
model_embedding.similar_by_vector(model_embedding['гена'])

[('гена', 1.0),
 ('крокодил', 0.6995565295219421),
 ('чебурашка', 0.6883164644241333),
 ('стас', 0.6753256916999817),
 ('виталик', 0.6509150862693787),
 ('михайлов', 0.6437711715698242),
 ('дружок', 0.6290872097015381),
 ('артём', 0.6183868646621704),
 ('вадим', 0.6158268451690674),
 ('никита', 0.6031651496887207)]

In [ ]:
model_embedding.most_similar(positive=['гена'])

[('крокодил', 0.6995565891265869),
 ('чебурашка', 0.6883165240287781),
 ('стас', 0.6753256916999817),
 ('виталик', 0.6509150862693787),
 ('михайлов', 0.6437711715698242),
 ('дружок', 0.6290872693061829),
 ('артём', 0.6183868646621704),
 ('вадим', 0.6158269047737122),
 ('никита', 0.6031651496887207),
 ('андрюха', 0.5998855233192444)]

In [ ]:
data = pd.read_csv('data_preprocessed.csv', converters={'text':ast.literal_eval})

In [ ]:
data

,Unnamed: 0,target,text,year
0,0,2,"[знать, получиться, очень, смешно, огонёк, нес...",1987
1,1,2,"[центр, сюжет, колоритный, компания, герой, по...",2019
2,2,2,"[повествовательный, критерий, нарратив, искате...",2022
3,3,2,"[хотеть, такой, отведать, добро, пожаловать, с...",2015
4,4,2,"[енотовидный, собака, приходиться, сражаться, ...",1994
...,...,...,...,...
1708,1708,0,"[«, любить, снова, », такой, тема, —, 2023г, п...",2023
1709,1709,0,"[действо, наполнить, тёмный, тусклый, мрачный,...",0
1710,1710,0,"[начинаться, безумно, затянутый, экспозиция, к...",2021
1711,1711,0,"[первый, треть, фильм, вполне, порадовать, при...",2023


In [ ]:
import string
punktuation = string.punctuation
punktuation+='«'+'»'+'—'

In [ ]:
def clean_punktuacy(x):
    return [word for word in x if not word in punktuation]

In [ ]:
data['text'] = data['text'].map(clean_punktuacy)

In [ ]:
data

,Unnamed: 0,target,text,year
0,0,2,"[знать, получиться, очень, смешно, огонёк, нес...",1987
1,1,2,"[центр, сюжет, колоритный, компания, герой, по...",2019
2,2,2,"[повествовательный, критерий, нарратив, искате...",2022
3,3,2,"[хотеть, такой, отведать, добро, пожаловать, с...",2015
4,4,2,"[енотовидный, собака, приходиться, сражаться, ...",1994
...,...,...,...,...
1708,1708,0,"[любить, снова, такой, тема, 2023г, переписка,...",2023
1709,1709,0,"[действо, наполнить, тёмный, тусклый, мрачный,...",0
1710,1710,0,"[начинаться, безумно, затянутый, экспозиция, к...",2021
1711,1711,0,"[первый, треть, фильм, вполне, порадовать, при...",2023


In [ ]:
def insert_digits(x):
    digits = [digit for digit in x if str(digit).isdigit()]

    return digits
def clean_digits(x):
    return [digit for digit in x if not str(digit).isdigit()]

In [ ]:
insert_digits([1, 'sdfsdf', 10])

[1, 10]

In [ ]:
data['digits'] = data['text'].map(insert_digits)
data['text'] = data['text'].map(clean_digits)

In [ ]:
data

,Unnamed: 0,target,text,year,digits
0,0,2,"[знать, получиться, очень, смешно, огонёк, нес...",1987,[10]
1,1,2,"[центр, сюжет, колоритный, компания, герой, по...",2019,"[8, 10]"
2,2,2,"[повествовательный, критерий, нарратив, искате...",2022,[]
3,3,2,"[хотеть, такой, отведать, добро, пожаловать, с...",2015,[10]
4,4,2,"[енотовидный, собака, приходиться, сражаться, ...",1994,[200]
...,...,...,...,...,...
1708,1708,0,"[любить, снова, такой, тема, 2023г, переписка,...",2023,[1]
1709,1709,0,"[действо, наполнить, тёмный, тусклый, мрачный,...",0,[10]
1710,1710,0,"[начинаться, безумно, затянутый, экспозиция, к...",2021,[]
1711,1711,0,"[первый, треть, фильм, вполне, порадовать, при...",2023,[10]


In [ ]:
mx = 0
mn = 1001010010
len_str = []
for i in range(1713):
    if len(data['text'][i]) < mn:
        mn = len(data['text'][i])
    if len(data['text'][i]) > mx:
        mx = len(data['text'][i])
    if len(data['text'][i]) == 1:
        print(data['text'][i])
        print(i)
    len_str.append(len(data['text'][i]))
mn, mx

['6,5']
1364


(0, 654)

In [ ]:
def normalize_len_text(x):
    if len(x) < 654:
        return x + ['стоп_слово'] * (654-len(x))
    return x

In [ ]:
data['text'] = data['text'].map(normalize_len_text)

In [ ]:
data

,Unnamed: 0,target,text,year,digits
0,0,2,"[знать, получиться, очень, смешно, огонёк, нес...",1987,[10]
1,1,2,"[центр, сюжет, колоритный, компания, герой, по...",2019,"[8, 10]"
2,2,2,"[повествовательный, критерий, нарратив, искате...",2022,[]
3,3,2,"[хотеть, такой, отведать, добро, пожаловать, с...",2015,[10]
4,4,2,"[енотовидный, собака, приходиться, сражаться, ...",1994,[200]
...,...,...,...,...,...
1708,1708,0,"[любить, снова, такой, тема, 2023г, переписка,...",2023,[1]
1709,1709,0,"[действо, наполнить, тёмный, тусклый, мрачный,...",0,[10]
1710,1710,0,"[начинаться, безумно, затянутый, экспозиция, к...",2021,[]
1711,1711,0,"[первый, треть, фильм, вполне, порадовать, при...",2023,[10]


In [ ]:
def sentence_to_vector(x):
    sequence_embeddings = np.zeros(100*110)
    count_words = 0
    count_words_to_sentense = 0
    for i in range(654):
        try:
            sequence_embeddings[count_words_to_sentense: count_words_to_sentense + 100] += model_embedding[x[i]]
        except:
            pass


        count_words += 1
        if count_words == 6:
            count_words = 0
            count_words_to_sentense += 100
    return sequence_embeddings / 10

In [ ]:
sentense = 'я смотрю фильм'.split()
s = np.zeros(200)
for i in range(3):
    s[100:] += model_embedding[sentense[i]]
s

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [ ]:
data['vectors'] = data['text'].map(sentence_to_vector)

In [ ]:
data

,Unnamed: 0,target,text,year,digits,vectors
0,0,2,"[знать, получиться, очень, смешно, огонёк, нес...",1987,[10],"[0.17411700189113616, -0.17218459034338593, -0..."
1,1,2,"[центр, сюжет, колоритный, компания, герой, по...",2019,"[8, 10]","[0.06799089834094048, 0.10005029700696469, -0...."
2,2,2,"[повествовательный, критерий, нарратив, искате...",2022,[],"[-0.1406810015439987, 0.01834691995754838, -0...."
3,3,2,"[хотеть, такой, отведать, добро, пожаловать, с...",2015,[10],"[-0.06202690489590168, 0.04860300123691559, -0..."
4,4,2,"[енотовидный, собака, приходиться, сражаться, ...",1994,[200],"[-0.09787630215287209, -0.009055999666452407, ..."
...,...,...,...,...,...,...
1708,1708,0,"[любить, снова, такой, тема, 2023г, переписка,...",2023,[1],"[0.1565872997045517, -0.09220499992370605, -0...."
1709,1709,0,"[действо, наполнить, тёмный, тусклый, мрачный,...",0,[10],"[-0.06905959621071815, -0.15049200057983397, -..."
1710,1710,0,"[начинаться, безумно, затянутый, экспозиция, к...",2021,[],"[0.19148579612374306, 0.029332999885082246, -0..."
1711,1711,0,"[первый, треть, фильм, вполне, порадовать, при...",2023,[10],"[0.06820570454001426, -0.11948579251766205, -0..."


In [ ]:
len(data['vectors'][0])

11000

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['vectors'], data['target'], test_size=0.2, random_state=42)

In [ ]:
def balancer(x, y):
    dataset = pd.DataFrame(x)
    dataset['target'] = y
    dataset = dataset.append(dataset[dataset['target'] == 1])
    dataset = dataset.append(dataset[dataset['target'] == 1])
    dataset = dataset.append(dataset[dataset['target'] == 0])
    dataset = dataset.append(dataset[dataset['target'] == 0])
    return dataset

df = balancer(X_train, y_train)
df

<ipython-input-27-44ad58d4eca6>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 1])
<ipython-input-27-44ad58d4eca6>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 1])
<ipython-input-27-44ad58d4eca6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 0])
<ipython-input-27-44ad58d4eca6>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 0])


,vectors,target
408,"[0.02564599961042404, 0.07915400117635726, -0....",2
1357,"[0.32533899545669553, 0.07317180167883634, -0....",1
518,"[0.15834299698472024, 0.10545659288764, -0.276...",2
1111,"[0.11678869761526585, 0.0015897981822490693, -...",1
538,"[-0.11294550001621247, -0.026350399106740953, ...",2
...,...,...
1685,"[0.0020008929073810577, -0.06789840115234255, ...",0
1687,"[0.25224399864673613, -0.058703719661571084, -...",0
1482,"[0.047589004039764404, -0.11227300018072128, -...",0
1638,"[0.1394219994544983, -0.020276400633156298, -0...",0


In [ ]:
X_train = np.stack(df['vectors'].values).round(2)
y_train = df.target

In [ ]:
len(data['vectors'][0])

11000

In [ ]:
def change_y(x):
    y_return = [0]*3
    y_return[x] = 1
    return np.array(y_return)

In [ ]:
y_train = df['target'].map(change_y)
y_train = np.stack(y_train.values)
y_train.shape

(2894, 3)

In [ ]:
X_train

array([[ 0.03,  0.08, -0.23, ...,  0.  ,  0.  ,  0.  ],
       [ 0.33,  0.07, -0.21, ...,  0.  ,  0.  ,  0.  ],
       [ 0.16,  0.11, -0.28, ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.05, -0.11, -0.12, ...,  0.  ,  0.  ,  0.  ],
       [ 0.14, -0.02, -0.1 , ...,  0.  ,  0.  ,  0.  ],
       [-0.05, -0.01, -0.3 , ...,  0.  ,  0.  ,  0.  ]])

In [ ]:
X_test = np.stack(X_test.values).round(2)
y_test = data['target'].map(change_y)
y_test = np.stack(y_test.values)
y_test

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [ ]:
model = keras.Sequential()

model.add(Dense(300, activation='tanh'))

model.add(Dense(200, activation='tanh'))

model.add(Dense(100, activation='tanh'))

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=[keras.metrics.Accuracy()])

In [ ]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
91/91 [==============================] - 6s 5ms/step - loss: 1.0711 - accuracy: 0.0000e+00
Epoch 2/100
91/91 [==============================] - 0s 4ms/step - loss: 0.5087 - accuracy: 0.0000e+00
Epoch 3/100
91/91 [==============================] - 0s 4ms/step - loss: 0.1715 - accuracy: 0.0000e+00
Epoch 4/100
91/91 [==============================] - 0s 5ms/step - loss: 0.1805 - accuracy: 0.0000e+00
Epoch 5/100
91/91 [==============================] - 0s 5ms/step - loss: 0.0775 - accuracy: 0.0000e+00
Epoch 6/100
91/91 [==============================] - 0s 5ms/step - loss: 0.0708 - accuracy: 0.0000e+00
Epoch 7/100
91/91 [==============================] - 0s 4ms/step - loss: 0.0395 - accuracy: 0.0000e+00
Epoch 8/100
91/91 [==============================] - 0s 5ms/step - loss: 0.0366 - accuracy: 0.0000e+00
Epoch 9/100
91/91 [==============================] - 0s 4ms/step - loss: 0.0359 - accuracy: 0.0000e+00
Epoch 10/100
91/91 [==============================] - 0s 4ms/step - loss:

In [ ]:
def max_index(x):
    for i in range(len(x)):
        if x[i] == max(x):
            return i

In [ ]:
def check_accuracy(y_pred, y_test):
    count_all = 0
    count_true = 0
    for i in range(len(y_pred)):
        if max_index(y_pred[i]) == max_index(y_test[i]):
            count_true += 1
        count_all += 1
    return count_true / count_all

In [ ]:
y_pred = model.predict(X_test)

11/11 [==============================] - 0s 3ms/step


In [ ]:
check_accuracy(y_pred, y_test)

0.8571428571428571

In [ ]:
X_test_work = np.array([[]])
X_test_work_common = np.array([[[]]])
for item in range(len(X_test)):
    for i in range(0, 500, 100):
        X_test_work = np.append(X_test_work, X_test[item][i:-(500-i)])
    X_test_work_common = np.append(X_test_work_common, X_test_work)
    X_test_work = np.array([[]])
X_test_work_common = X_test_work_common.reshape(len(X_test), 5, 10500)

In [ ]:
X_train_work = np.array([[]])
X_train_work_common = np.array([[[]]])
for item in range(len(X_train)):
    for i in range(0, 500, 100):
        X_train_work = np.append(X_train_work, X_train[item][i:-(500-i)])
    X_train_work_common = np.append(X_train_work_common, X_train_work)
    X_train_work = np.array([[]])
X_train_work_common = X_train_work_common.reshape(len(X_train), 5, 10500)

In [ ]:
model_rnn = keras.Sequential()



model_rnn.add(LSTM(1024, activation='tanh', return_sequences=True))
model_rnn.add(LSTM(1024, activation='tanh', return_sequences=False))
model_rnn.add(Dense(1024, activation='tanh'))
model_rnn.add(Dense(512, activation='tanh'))
model_rnn.add(Dense(256, activation='tanh'))
model_rnn.add(Dense(3, activation='softmax'))

model_rnn.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001))

In [ ]:
model_rnn.fit(X_train_work_common, y_train, epochs=15)

Epoch 1/15
91/91 [==============================] - 12s 42ms/step - loss: 1.1958
Epoch 2/15
91/91 [==============================] - 4s 42ms/step - loss: 1.0741
Epoch 3/15
91/91 [==============================] - 4s 41ms/step - loss: 0.8991
Epoch 4/15
91/91 [==============================] - 4s 41ms/step - loss: 0.5022
Epoch 5/15
91/91 [==============================] - 4s 42ms/step - loss: 0.2704
Epoch 6/15
91/91 [==============================] - 4s 41ms/step - loss: 0.1380
Epoch 7/15
91/91 [==============================] - 4s 42ms/step - loss: 0.1175
Epoch 8/15
91/91 [==============================] - 4s 43ms/step - loss: 0.0973
Epoch 9/15
91/91 [==============================] - 4s 42ms/step - loss: 0.0643
Epoch 10/15
91/91 [==============================] - 4s 41ms/step - loss: 0.0397
Epoch 11/15
91/91 [==============================] - 4s 42ms/step - loss: 0.0368
Epoch 12/15
91/91 [==============================] - 4s 42ms/step - loss: 0.0355
Epoch 13/15
91/91 [=================

In [ ]:
y_pred_rnn = model_rnn.predict(X_test_work_common)

11/11 [==============================] - 1s 18ms/step


In [ ]:
y_pred_rnn

array([[6.70961117e-06, 5.49156265e-03, 9.94501710e-01],
       [4.28602689e-05, 6.47969835e-04, 9.99309182e-01],
       [3.46375033e-02, 1.10213601e-04, 9.65252280e-01],
       ...,
       [2.68341228e-05, 1.06415555e-05, 9.99962568e-01],
       [1.32621161e-03, 4.15641625e-05, 9.98632252e-01],
       [3.90439491e-05, 5.91223361e-05, 9.99901772e-01]], dtype=float32)

In [ ]:
check_accuracy(y_pred_rnn, y_test)

0.8571428571428571

In [ ]:
model_rnn.save('Model_rnn_10epochs_512x512x3')

In [ ]:
def sentense_to_vector_by_word(sentense):
